In [8]:
# Import SparkSession
import findspark
findspark.init()

from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.sql import SparkSession
import random
import sys
#path = str(sys.argv[1])
#atr = int(sys.argv[2])

In [9]:
# Build the SparkSession
spark = SparkSession.builder \
    .master("local[6]") \
    .appName("Data exploration URL") \
    .config("spark.executor.memory", "6gb") \
    .getOrCreate()

sc = spark.sparkContext

In [10]:
sc._conf.getAll()

[('spark.driver.memory', '6g'),
 ('spark.rdd.compress', 'True'),
 ('spark.driver.port', '44733'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.executor.memory', '6gb'),
 ('spark.app.name', 'Data exploration URL'),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.id', 'local-1614703368308'),
 ('spark.master', 'local[6]'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.host', 'fedora')]

In [11]:
# Load training data
data = spark.read.format("libsvm")\
    .load("../data/url_svmlight/prueba_10.svm")
# Split the data into train and test
seed = random.randrange(500, 1300, 2)
splits = data.randomSplit([0.6, 0.4], seed)

train = splits[0]
test = splits[1]

In [12]:
data.take(1)

[Row(label=0.0, features=SparseVector(3209930, {3: 0.0581, 4: 0.0897, 5: 0.1765, 10: 0.1429, 20: 0.2857, 21: 0.0238, 23: 1.0, 24: 1.0, 27: 1.0, 35: 1.0, 43: 1.0, 53: 1.0, 55: 1.0, 61: 1.0, 63: 1.0, 65: 1.0, 67: 1.0, 69: 1.0, 71: 1.0, 73: 1.0, 75: 1.0, 81: 1.0, 83: 1.0, 85: 1.0, 87: 1.0, 89: 1.0, 91: 1.0, 93: 1.0, 95: 1.0, 101: 1.0, 103: 1.0, 105: 1.0, 107: 1.0, 109: 1.0, 111: 1.0, 128: 1.0, 150: 1.0, 152: 1.0, 154: 1.0, 711: 1.0, 712: 1.0, 713: 1.0, 714: 1.0, 715: 1.0, 716: 1.0, 1281: 1.0, 1282: 1.0, 1283: 1.0, 1284: 1.0, 1285: 1.0, 1286: 1.0, 3482: 1.0, 4121: 1.0, 8156: 1.0, 155152: 1.0, 155153: 1.0, 155154: 1.0, 155155: 1.0, 155156: 1.0, 155162: 1.0, 155163: 1.0, 155165: 1.0, 155167: 1.0, 155168: 1.0, 155169: 1.0, 155171: 1.0, 155172: 1.0, 155173: 1.0, 155174: 1.0, 155175: 1.0, 155176: 1.0, 155177: 1.0, 155178: 1.0, 155179: 1.0, 155180: 1.0, 155181: 1.0, 155182: 1.0, 155193: 1.0, 155194: 1.0, 155195: 1.0, 155196: 1.0, 155197: 1.0, 155198: 1.0, 155199: 1.0, 155200: 1.0, 155201: 1.0, 1

In [13]:
# specify layers for the neural network:
# input layer of size 4 (features), two intermediate of size 5 and 4
# and output of size 3 (classes)
layers = [3209930, 5, 4, 2]

# create the trainer and set its parameters
trainer = MultilayerPerceptronClassifier(
    maxIter=100, layers=layers, blockSize=10000, seed=1234)

# train the model
model = trainer.fit(train)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Exception ignored in: <function JavaObject.__init__.<locals>.<lambda> at 0x7fc9ccca61f0>
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1341, in <lambda>
KeyboardInterrupt: 
Exception ignored in: <function JavaObject.__init__.<locals>.<lambda> at 0x7fc9ccb97670>
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1341, in <lambda>
  File "/opt/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 645, in _garbage_collect_object
  File "/opt/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 964, in garbage_collect_object
  File "/opt/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1031, in send_command
  File "/opt/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 979, in _get_connection
  File "/opt/

In [ ]:
# compute accuracy on the test set
result = model.transform(test)
predictionAndLabels = result.select("prediction", "label")

In [ ]:
evaluator = MulticlassClassificationEvaluator(metricName="f1")
print("f1: " + str(evaluator.evaluate(predictionAndLabels)))
evaluator = MulticlassClassificationEvaluator(metricName="weightedPrecision")
print("weightedPrecision: " + str(evaluator.evaluate(predictionAndLabels)))
evaluator = MulticlassClassificationEvaluator(metricName="weightedRecall")
print("weightedRecall: " + str(evaluator.evaluate(predictionAndLabels)))
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Accuracy: " + str(evaluator.evaluate(predictionAndLabels)))

In [1]:
x = 1

In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql.types import StringType
from pyspark import SQLContext
from pyspark.sql import functions as F
conf = SparkConf().setMaster("local[11]").setAppName("KDD")
conf.set("spark.driver.memory", "28g") 
sc=SparkContext(conf=conf)
sqlContext=SQLContext(sc)

In [2]:
dfspark=sqlContext.read.format("csv").option("header","true").option("inferSchema","true").load('../data/url_svmlight/prueba.svm')

In [ ]:
dfspark.head()

In [ ]:
dfspark.printSchema()